In [1]:
import os

import numpy as np

import asdf


np.random.seed(42)

def print_file(fn):
    """
    A helper function to print out an ASDF file
    """
    with open(fn, "r", encoding="unicode_escape") as f:
        print(f.read())

## Exercise 1
Create an [AsdfFile](https://asdf.readthedocs.io/en/latest/api/asdf.AsdfFile.html#asdf.AsdfFile) instance and build a tree containing all of the above supported types.

In [2]:
af = asdf.AsdfFile()
af.tree = {
    "my_dict": {"a": 1, "b": 2},
    "my_list": [1, 2, 3],
    "my_string": "abc",
    "my_int": 1,
    "my_float": 1.1,
    "my_none": None,
}
af.info()

root (AsdfObject)
├─my_dict (dict)
│ ├─a (int): 1
│ └─b (int): 2
├─my_list (list)
│ ├─[0] (int): 1
│ ├─[1] (int): 2
│ └─[2] (int): 3
├─my_string (str): abc
├─my_int (int): 1
├─my_float (float): 1.1
└─my_none (NoneType): None


## Exercise 2
Recreate (if necessary) your custom tree containing all of the supported types and write it to an ASDF file. Open the file in a text editor and view the contents.

In [3]:
af.write_to("my_file.asdf")
print_file("my_file.asdf")

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 3.5.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/core/extensions/core-1.5.0
    manifest_software: !core/software-1.0.0 {name: asdf_standard, version: 1.1.1}
    software: !core/software-1.0.0 {name: asdf, version: 3.5.0}
my_dict: {a: 1, b: 2}
my_float: 1.1
my_int: 1
my_list: [1, 2, 3]
my_none: null
my_string: abc
...



## Exercise 3
Open one of the ASDF files created above. What is the type of value stored with the "asdf_library" library in the tree?

In [6]:
af = asdf.open("my_file.asdf")
print(type(af["asdf_library"]))

<class 'asdf.tags.core.Software'>


## Exercise 4
Generate an ASDF file with 3 arrays and save it to disk. Examine the file contents.

In [10]:
af = asdf.AsdfFile()
af["arrays"] = [
    np.arange(42),
    np.zeros(10, dtype=np.float32),
    np.ones((20, 30, 40), dtype=np.uint8),
]
af.write_to("arrays.asdf")
print_file("arrays.asdf")

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 3.5.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/core/extensions/core-1.5.0
    manifest_software: !core/software-1.0.0 {name: asdf_standard, version: 1.1.1}
    software: !core/software-1.0.0 {name: asdf, version: 3.5.0}
arrays:
- !core/ndarray-1.0.0
  source: 0
  datatype: int64
  byteorder: little
  shape: [42]
- !core/ndarray-1.0.0
  source: 1
  datatype: float32
  byteorder: little
  shape: [10]
- !core/ndarray-1.0.0
  source: 2
  datatype: uint8
  byteorder: big
  shape: [20, 30, 40]
...
ÓBLK 0              P      P      P¤=ç;qóÀfCàkFíÈ                                                               	       
                 

## Exercise 5
Save an ASDF file with a large array and a small view of the array. Open this file and change the view contents. This will require disabling memory mapping by passing `memmap=False` to [asdf.open](https://asdf.readthedocs.io/en/latest/api/asdf.open.html#asdf.open). What happens to the large array?

In [24]:
af = asdf.AsdfFile()
af["big"] = np.zeros((100, 100), dtype=np.uint16)
af["view"] = af["big"][:10, :10]
af.write_to("view.asdf")
print_file("view.asdf")

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 3.5.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/core/extensions/core-1.5.0
    manifest_software: !core/software-1.0.0 {name: asdf_standard, version: 1.1.1}
    software: !core/software-1.0.0 {name: asdf, version: 3.5.0}
big: !core/ndarray-1.0.0
  source: 0
  datatype: uint16
  byteorder: little
  shape: [100, 100]
view: !core/ndarray-1.0.0
  source: 0
  datatype: uint16
  byteorder: little
  shape: [10, 10]
  strides: [200, 2]
...
ÓBLK 0              N       N       N ²þEsHÞ$:WÓQp                                                                                                                                                                

In [25]:
af = asdf.open("view.asdf", memmap=False)
af["view"][:] = 1
print(af["big"][:])

[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Exercise 6
Generate an ASDF file with:
- one array compressed with "zlib"
- a second array that is uncompressed

What happens if you read and then rewrite the file to a new filename?

In [28]:
af = asdf.AsdfFile()
af["uncompressed"] = np.arange(42)
af["compressed"] = np.zeros(100)
af.set_array_compression(af["compressed"], "zlib")
af.write_to("compressed.asdf")
print_file("compressed.asdf")

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 3.5.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/core/extensions/core-1.5.0
    manifest_software: !core/software-1.0.0 {name: asdf_standard, version: 1.1.1}
    software: !core/software-1.0.0 {name: asdf, version: 3.5.0}
compressed: !core/ndarray-1.0.0
  source: 1
  datatype: float64
  byteorder: little
  shape: [100]
uncompressed: !core/ndarray-1.0.0
  source: 0
  datatype: int64
  byteorder: little
  shape: [42]
...
ÓBLK 0              P      P      P¤=ç;qóÀfCàkFíÈ                                                               	       
                     
                                                               

In [30]:
af = asdf.open("compressed.asdf")
print(af.get_array_compression(af["compressed"]))
af.write_to("compressed_copy.asdf")

zlib


In [33]:
af = asdf.open("compressed_copy.asdf")
print(af.get_array_compression(af["compressed"]))

zlib


## Exercise 7
Write an ASDF file containing the following `astropy` objects:
1. [Quantity](https://docs.astropy.org/en/stable/units/quantity.html)
2. A [model](https://docs.astropy.org/en/stable/api/astropy.modeling.Model.html#astropy.modeling.Model)

   Hint: The [astropy.modeling](https://docs.astropy.org/en/stable/modeling/index.html) package provides a framework for representing models and performing model evaluation and fitting. Models are initialized using their parameters like in the following example for [Gaussian1D](https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian1D.html#astropy.modeling.functional_models.Gaussian1D):
   ```
   from astropy.modeling import models
   gauss = models.Gaussian1D(amplitude=10, mean=3, stddev=1.2)
   ```
3. A [Time](https://docs.astropy.org/en/stable/time/index.html) object

    Hint: The [astropy.time](https://docs.astropy.org/en/stable/time/ref_api.html#module-astropy.time) package provides functionality for manipulating times and dates. To initialize it supply a string and a [format](https://docs.astropy.org/en/stable/time/index.html#id3), or supply a datetime object.
    
4. A [ICRS](https://docs.astropy.org/en/stable/api/astropy.coordinates.ICRS.html) coordinate object.

In [42]:
import astropy.units as u
from astropy.modeling import models
from astropy import coordinates, modeling, time

af = asdf.AsdfFile()
af["quantity"] = u.Quantity([1, 2, 3], u.meter)
af["qaussian"] = models.Gaussian1D(amplitude=10, mean=3, stddev=1.2)
af["time"] = time.Time.now()
af["coordinate"] = coordinates.ICRS(ra=0 * u.deg, dec=0 * u.deg)
af.write_to("astropy_objects.asdf")
print_file("astropy_objects.asdf")

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 3.5.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/core/extensions/core-1.5.0
    manifest_software: !core/software-1.0.0 {name: asdf_standard, version: 1.1.1}
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.6.1}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/transform/extensions/transform-1.5.0
    manifest_software: !core/software-1.0.0 {name: asdf_transform_schemas, version: 0.5.0}
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.6.1}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest